# 기본 라이브러리 로드

In [1]:
import time
import pandas as pd
import numpy as np


from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


import warnings
warnings.filterwarnings(action = 'ignore')


# 함수

In [2]:
def try_none_text(x):
    try:
        return x.text
    except:
        return 'NA'
        

In [3]:
def try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        return wd.find_element_by_class_name(class_name).text
    except NoSuchElementException:
        print(col, end=' ')
        return 'NA'
        

In [4]:
def strong_try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        temps = wd.find_elements_by_class_name(class_name)
        for temp in temps:
            if temp.text.startswith('편의'):
                return temp.text.split('\n')[1]
        else:
            return 'NA'
    except NoSuchElementException:
        print(col, end=' ')
        return 'NA'
        

## 가게 스크래핑 함수

In [5]:
def get_store_info(wd):
    name = wd.find_element_by_class_name('_3XamX').text
#     print(name)
    address = try_none_text_by_class_name(wd,'_2yqUQ','address')
#     print(address)
    tel = try_none_text_by_class_name(wd,'_3ZA0S','tel') 
#     print(tel)
    detail = try_none_text_by_class_name(wd,'WoYOw','detail')
#     print('d1',detail)
    strong = strong_try_none_text_by_class_name(wd,'_1M_Iz.undefined','strong')
#     strong = try_none_text_by_selector(wd, '#app-root > div > div > div.place_detail_wrapper > div:nth-child(4) > div > div:nth-child(3) > div > ul > li._1M_Iz.undefined > div','strong')
#     print('s1',strong)

    try:
        datalab_1 = {}
        datalab_list = wd.find_elements_by_class_name('_3Ryhx')
        for datalab in datalab_list:
            datalab_1[datalab.find_element_by_class_name('_3hvd9').text] = [x.text for x in datalab.find_elements_by_class_name('_2irYJ') if x.text != '']
    except:
        datalab_1 = 'NA'
        
#     print(name, datalab_1)
        
        

    nums_list = wd.find_elements_by_class_name('_1Y6hi')
    score = nums_list[0].find_element_by_tag_name('em').text
    review_num = nums_list[1].find_element_by_tag_name('em').text
    blog_nums = nums_list[2].find_element_by_tag_name('em').text
#     print(score, review_num, blog_nums)

    return name, address,tel, score, detail, strong, datalab_1, review_num, blog_nums


## 댓글 스크래핑 함수

## 스크래핑 함수

In [6]:
구이름 = '송파구'
동이름 = '방이동'
업종 = '식당'

In [7]:
search_url = 'https://map.naver.com/v5/search/'
search_url_detail = search_url + 구이름 +'%20'+ 동이름 + '%20' + 업종
print(search_url_detail)

https://map.naver.com/v5/search/송파구%20방이동%20식당


In [ ]:
options = webdriver.ChromeOptions()
# options.add_argument('--no-sandbox')

options.add_argument('headless')
wd = webdriver.Chrome('chromedriver.exe')  # chromedriver 열기
wd.implicitly_wait(3)

wd.execute_script('window.open("about:blank", "_blank");')
tabs = wd.window_handles

wd.switch_to.window(tabs[0])

wd.get(search_url_detail)
time.sleep(1)
store_idx = 0
store_df = pd.DataFrame(columns=("name","address","tel","score","detail","strong","datalab_1","review_num","blog_num"))
review_idx = 0
review_df = pd.DataFrame()


start_time = time.time()

print(f'{구이름} {동이름} {업종} 시작합니다~~ 후비고~~')

time.sleep(2)

wd.switch_to_frame('searchIframe')
# print('ch')

while True:
    # 스크롤 다운(iframe)
    
    dfd = wd.find_element_by_tag_name('body')
    dfd.click()
    for _ in range(7):
        dfd.send_keys(Keys.END)
        time.sleep(0.3)

        
    time.sleep(2)
    
    # 가게 링크
    inline_list = wd.find_elements_by_class_name('Ow5Yt')
    print(len(inline_list))
    
    # 각각의 가게 정보 스크랩핑
    for inline in inline_list:
        try:
            inline.click()
        except ElementClickInterceptedException:
            print('Ele_C_Ex')
            pass
        time.sleep(0.8)

        # iframe 전환
        wd.switch_to_default_content()
        wd.switch_to_frame('entryIframe')    

        store_df.loc[store_idx] = get_store_info(wd)
        print(store_idx, end='#')
        store_idx += 1

        
#         리뷰
#         review_df.loc[review_idx] = (name, date, score, comment)
#         review_idx += 1







        # iframe전환
        wd.switch_to_default_content()
        wd.switch_to_frame('searchIframe')
    else:
        print('f{current_num}page done', end= ' ')
        
    last_num = wd.find_elements_by_class_name('_2tk2s ')[-1].text
    current_num = wd.find_element_by_class_name('_5vmWW').text
    
    if last_num == current_num:
        print(구이름,동이름,업종, f'가게 {store_idx}개, 리뷰 {review_idx}  완료' )
        print(f'{start_time-time.time()} 소요')
        break
    
    try:
        next_bnt = wd.find_element_by_css_selector('#app-root > div > div._2lx2y > div._2ky45 > a:nth-child(7) > svg')
        next_bnt.click()
        time.sleep(1)
        
    except:
        print('page button 에러')
        break

    print(current_num,last_num)
    

송파구 방이동 식당 시작합니다~~ 후비고~~
ch
50
s1 단체석, 주차, 포장, 배달, 예약, 무선 인터넷, 남/녀 화장실 구분, 국민지원금
0#s1 단체석
1#s1 단체석, 포장, 배달, 무선 인터넷, 남/녀 화장실 구분, 국민지원금
2#s1 단체석, 주차, 포장, 무선 인터넷, 남/녀 화장실 구분, 국민지원금
3#s1 단체석, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분, 국민지원금
4#s1 단체석, 발렛파킹, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분
5#s1 예약, 무선 인터넷, 제로페이, 국민지원금
6#s1 포장, 예약, 무선 인터넷, 남/녀 화장실 구분, 국민지원금
7#s1 단체석, 주차, 포장, 배달, 예약, 무선 인터넷, 반려동물 동반, 남/녀 화장실 구분, 지역화폐(카드형), 지역화폐(모바일형), 제로페이, 국민지원금
8#s1 단체석, 예약, 무선 인터넷, 남/녀 화장실 구분
9#s1 단체석, 발렛파킹, 예약
10#s1 단체석, 발렛파킹, 포장, 남/녀 화장실 구분, 장애인 편의시설
11#s1 포장, 배달, 무선 인터넷, 국민지원금
12#s1 무선 인터넷, 남/녀 화장실 구분, 지역화폐(카드형), 제로페이, 국민지원금
13#s1 단체석, 예약, 무선 인터넷


In [ ]:
store_df